In [4]:
import pandas as pd


In [5]:
df = pd.read_csv('form.csv')

print(df) 

            Timestamp            Name Are you visiting for the first time?  \
0   3/31/2025 9:28:48      Rebecca Lu                                  NaN   
1    4/1/2025 7:57:16    Derek Liang                                   NaN   
2    4/1/2025 7:57:32     Logan golia                                  NaN   
3    4/1/2025 8:04:38     Daniel Kim                                   NaN   
4    4/1/2025 8:09:57      Kyle Hwang                                  NaN   
..                ...             ...                                  ...   
62  4/4/2025 22:08:58     Daniel Choi                                  NaN   
63  4/4/2025 22:35:19      audrey lee                                  NaN   
64  4/4/2025 22:38:41       Brian Seo                                  NaN   
65  4/4/2025 23:18:55  Nathanael Wang                                  NaN   
66                NaN      Jiwang Lee                                  NaN   

    House Church              Where would you like to be picked

In [122]:
class Driver:
    def __init__(self, name, amount_seats, pickup_location, service_type, plans):
        self.name = name
        self.amount_seats = amount_seats
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name


class Rider:
    def __init__(self, name, pickup_location, service_type, plans):
        self.name = name
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

In [140]:
people_list = df["Name"]
locations_list = df["Where would you like to be picked up?"]
service_list = df["Which service are you attending?"]
is_driver_list = df["Are you a driver?"]
plans_list = df["Preferred after church plans?"]

drivers = set()
riders = set()

for i in range(len(people_list)):
    # p = (is_driver_list[i])
    # print(type(p))
    if is_driver_list[i] == "Yes!":
        # print(is_driver_list[i])
        drivers.add(Driver(people_list[i], 4, locations_list[i], service_list[i], plans_list[i]))
    else:
        riders.add(Rider(people_list[i], locations_list[i], service_list[i], plans_list[i]))

# print("List of all drivers:\n")
# for d in drivers:
#     print(d.name)

In [141]:
'''
Change amount of seats from the default number of 4 to another amount.
'''
for driver in drivers:
    if driver.name == "Ellen Kang":
        driver.amount_seats = 6

In [142]:
'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders = {
    "Josh Paik": {"Jacob Lei"},
    "Ric Chang": {"Karina Pan"},
    "Oriana Tang": {"Grace Park"}
}

'''
Assign riders who should ride together
'''
rider_groups = [
    {"Joann", "Grace Sowon Park "},
]



In [143]:
def print_assignment(assignments): # driver to list of rider object
    for driver, riders in assignments.items():
        print(f"\nDriver: {driver.name} {driver.pickup_location.split()[0]}")
        for rider in riders:
            print(rider.name, rider.pickup_location.split()[0])

In [144]:
# Amount of distinct locations drivers should have to pick up from
MAXIMUM_LOCATION_THRESHOLD = 2

In [145]:
from collections import defaultdict
from itertools import combinations

def assign_riders_greedy(drivers, riders):
    print(len(riders))
    assignments = defaultdict(list)
    unassigned_riders = set(list(riders))

    # Group drivers and riders by pickup time
    riders_by_time = defaultdict(list)
    for rider in riders:
        riders_by_time[rider.service_type].append(rider)
    
    driver_name_to_object = dict()
    for driver in drivers:
        driver_name_to_object[driver.name] = driver

    rider_name_to_object = dict()
    for rider in riders:
        rider_name_to_object[rider.name] = rider

    # Assign passengers who need to ride with a specific driver
    for driver_name, required_rider_names in driver_required_riders.items():
        time = driver_name_to_object[driver_name].service_type
        possible_riders = riders_by_time[time]
        for rider_name in required_rider_names:
            rider = rider_name_to_object[rider_name]
            driver = driver_name_to_object[driver_name]
            if rider in unassigned_riders:
                assignments[driver].append(rider)
                unassigned_riders.remove(rider)
                driver.amount_seats -= 1
        # print_assignment(assignments)

    
    # Assign groups of riders.
    # List of list of riders objects
    rider_groups_obs = []
    for group in rider_groups:
        group_obs = [rider_name_to_object[rider_name] for rider_name in group]
        rider_groups_obs.append(group_obs)

    for group in rider_groups_obs:
        if all(r in unassigned_riders for r in group):
            for driver in drivers:
                if driver.amount_seats < len(group):
                    continue

                combined_group = assignments[driver] + group
                total_locations = {r.pickup_location for r in combined_group}
                if len(total_locations) <= MAXIMUM_LOCATION_THRESHOLD:
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    driver.amount_seats -= len(group)
                    break


    
    # Greedily assign passengers to drivers.
    for driver in drivers:
        print(f'\nDriver {driver.name}')

        time = driver.service_type
        possible_riders = riders_by_time[time]

        found_set = False
        # Try forming groups of the amount that the passenger can hold all the way down to 0 passengers.
        for group_size in range(driver.amount_seats, 0, -1):
            if found_set:
                break
            for group in combinations(possible_riders, group_size):
                combined_group = list(group) + assignments[driver]
                locations = {r.pickup_location for r in combined_group} 
                if len(locations) <= MAXIMUM_LOCATION_THRESHOLD and all(r in unassigned_riders for r in group):
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    found_set = True
                    break  # Move to next driver
    print(f'unassigned riders: {[rider.name for rider in unassigned_riders]}')
    return assignments
x = assign_riders_greedy(drivers, riders)
# print(x)

52

Driver Jacob Deadlifts

Driver Jack Havemann

Driver Matthew Ahn

Driver Daniel Choi

Driver Olivia Chang

Driver Brian Seo

Driver Lia Kim

Driver Josh Paik

Driver Jonathan Mak

Driver Oriana Tang

Driver Clay Murphy

Driver Jocelyn Lee

Driver Ric Chang

Driver Ellen Kang
unassigned riders: []


In [146]:
print_assignment(x)



Driver: Josh Paik Off
Jacob Lei Off
Josh Yang Off
Sam Ko  Off
Kyle Hwang South

Driver: Ric Chang Off
Karina Pan Off
Elise Cho South
Derek Liang  South
Jeff Jiang Off

Driver: Oriana Tang Off
Grace Park Life

Driver: Jacob Deadlifts Off
Joann South
Grace Sowon Park  South
Susanna Tang North
Emily Yang North

Driver: Jack Havemann Life
Logan golia North
Benjamin Kim South
David Zhu South
helena song North

Driver: Matthew Ahn Off
Aaron Long Life
Nathanael Wang Life
Ella lu South
Hannah Zhang South

Driver: Daniel Choi Off
Jay Jung North
chloe lim Life
Ellie Jung  Life
Gabriel Ni North

Driver: Olivia Chang Life
Jeffery Huang South
Sehyun Jung Life
Taeho Choe  South
Phillip Seo Life

Driver: Brian Seo South
Grace Kwon South
Joshua Lee Life
Daniel Kuo South
Christina Ko Life

Driver: Lia Kim Off
audrey lee South
Rachel Kim South
Jane Yoo Off

Driver: Jonathan Mak North
JJ Lee Life
Claire Doh South
Hannah Kim South
Ethan Yu Life

Driver: Clay Murphy Life
Rebecca Lu Off
Darius Ajebon  Nort